In [1]:
import torch
import torchvision

In [2]:
import torch.nn as nn
import torch.nn.functional as F

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(
            in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out
    
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 128, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.linear = nn.Linear(256*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

In [7]:
model = ResNet(BasicBlock, [1, 2, 2, 2])
sum(p.numel() for p in model.parameters() if p.requires_grad)

5369162

In [4]:
GPU_BATCH_SIZE_LIMIT = 512

def train(model, criterion, optimizer, dataloader, device, scheduler = None, use_scheduler = False, batch_size = 32):
    model.train()
    train_loss = 0
    correct = 0
    count = 0
    total = 0
    for i, data in enumerate(dataloader, 0):
        image, label = data
        image = image.to(device)
        label = label.to(device)
        
        if batch_size >= GPU_BATCH_SIZE_LIMIT:
            if(count == 0):
                optimizer.step()
                optimizer.zero_grad()
                count = batch_size/32

            output = model(image)

            loss = criterion(output, label)*32/batch_size
            loss.backward()
            count = count - 1

            pred = torch.max(output.data, 1)[1]
            
            cur_correct = (pred == label).sum().item()
            cur_loss = loss.item()
        
        else:
            optimizer.zero_grad()
            output = model(image)
            loss = criterion(output, label)

            train_loss += loss.item()

            pred = torch.max(output.data, 1)[1]
            cur_correct = (pred == label).sum().item()
            cur_loss = loss.item()

            loss.backward()
            optimizer.step()
        
        total += label.size(0)
        correct += cur_correct
        train_loss += cur_loss

        if use_scheduler:
            scheduler.step()

    accuracy = correct/total
    train_loss = train_loss/len(dataloader)

    return train_loss, accuracy

def test(model, criterion, dataloader, device, batch_size = 32):
    model.eval()
    test_loss = 0
    correct = 0
    count = 0
    total = 0
    for i, data in enumerate(dataloader, 0):
        image, label = data
        image = image.to(device)
        label = label.to(device)
                
        if batch_size >= GPU_BATCH_SIZE_LIMIT:
            if(count == 0):
                count = batch_size/32

            output = model(image)

            loss = criterion(output, label)*32/batch_size
            count = count - 1

            pred = torch.max(output.data, 1)[1]
            
            cur_correct = (pred == label).sum().item()
            cur_loss = loss.item()
        
        else:
            output = model(image)
            loss = criterion(output, label)

            pred = torch.max(output.data, 1)[1]
            cur_correct = (pred == label).sum().item()
            cur_loss = loss.item()
            
        total += label.size(0)
        correct += cur_correct
        test_loss += cur_loss

    accuracy = correct/total
    test_loss = test_loss/len(dataloader)

    return test_loss, accuracy

In [15]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'


In [9]:
import torch
import torch.nn as nn

# Define the BasicBlock class
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_channels, out_channels, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.stride = stride

    def forward(self, x):
        identity = x

        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)

        x = self.conv2(x)
        x = self.bn2(x)

        if self.stride != 1 or identity.size(1) != x.size(1):
            identity = F.avg_pool2d(identity, 2, self.stride)
            identity = torch.cat((identity, torch.zeros(identity.size(0), x.size(1) - identity.size(1), identity.size(2), identity.size(3)).to(x.device)), dim=1)

        x += identity
        x = self.relu(x)

        return x

# Define the ResNet-14 model for CIFAR-10
class ResNet14(nn.Module):
    def __init__(self, num_classes=10):
        super(ResNet14, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self._make_layer(32, 32, 2)
        self.layer2 = self._make_layer(32, 64, 2, stride=2)
        self.layer3 = self._make_layer(64, 128, 2, stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(128, num_classes)

    def _make_layer(self, in_channels, out_channels, num_blocks, stride=1):
        layers = []
        layers.append(BasicBlock(in_channels, out_channels, stride))
        for _ in range(1, num_blocks):
            layers.append(BasicBlock(out_channels, out_channels, stride=1))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

# Create an instance of ResNet-14 model for CIFAR-10
model = ResNet14(num_classes=10)
sum(p.numel() for p in model.parameters() if p.requires_grad)

686026

In [31]:
model = ResNet(BasicBlock, [1, 2, 1, 2])
sum(p.numel() for p in model.parameters() if p.requires_grad)

5198666

In [18]:
def train_val(model, criterion, optimizer, train_loader, val_loader, device, scheduler = None, use_scheduler = False):
    model.train()
    train_loss = 0
    correct = 0
    count = 0
    total = 0
    for i, data in enumerate(train_loader, 0):
        image, label = data
        image = image.to(device)
        label = label.to(device)
    
        optimizer.zero_grad()
        output = model(image)
        loss = criterion(output, label)

        train_loss += loss.item()

        pred = torch.max(output.data, 1)[1]
        cur_correct = (pred == label).sum().item()
        cur_loss = loss.item()

        loss.backward()

        optimizer.step()
        
        total += label.size(0)
        correct += cur_correct
        train_loss += cur_loss

    train_accuracy = correct/total
    train_loss = train_loss/len(train_loader)
    
    model.eval()
    valid_loss = 0
    correct = 0
    count = 0
    total = 0
    for i, data in enumerate(val_loader, 0):
        image, label = data
        image = image.to(device)
        label = label.to(device)
                
        output = model(image)
        loss = criterion(output, label)

        pred = torch.max(output.data, 1)[1]
        cur_correct = (pred == label).sum().item()
        cur_loss = loss.item()
            
        total += label.size(0)
        correct += cur_correct
        valid_loss += cur_loss

    valid_accuracy = correct/total
    valid_loss = valid_loss/len(val_loader)
    
    if use_scheduler:
        scheduler.step(valid_loss)

    return train_loss, train_accuracy, valid_loss, valid_accuracy

def test(model, criterion, dataloader, device):
    model.eval()
    test_loss = 0
    correct = 0
    count = 0
    total = 0
    for i, data in enumerate(dataloader, 0):
        image, label = data
        image = image.to(device)
        label = label.to(device)
                
        output = model(image)
        loss = criterion(output, label)

        pred = torch.max(output.data, 1)[1]
        cur_correct = (pred == label).sum().item()
        cur_loss = loss.item()
            
        total += label.size(0)
        correct += cur_correct
        test_loss += cur_loss

    accuracy = correct/total
    test_loss = test_loss/len(dataloader)

    return test_loss, accuracy

EXPERIMENT 1

In [19]:
batch_size = 32

transform_train = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

transform_test = torchvision.transforms.Compose([
                    torchvision.transforms.ToTensor(), 
                    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

trainset = torchvision.datasets.CIFAR10(root = './data', train = True, download = True, transform = transform_train)

# Split the train data into train and validation sets
train_size = int(0.8 * len(trainset))
val_size = len(trainset) - train_size
trainset, valset = torch.utils.data.random_split(trainset, [train_size, val_size])

testset = torchvision.datasets.CIFAR10(root = './data', train = False, download = True, transform = transform_test)

train_loader = torch.utils.data.DataLoader(trainset, batch_size = batch_size, shuffle = True)
val_loader   = torch.utils.data.DataLoader(valset, batch_size = batch_size, shuffle = True)
test_loader  = torch.utils.data.DataLoader(testset, batch_size = batch_size, shuffle = False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

model = ResNet14(num_classes=10)
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Num Params: {num_params}\n")
model.to(device)
criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(model.parameters(), 0.1, momentum=0.9, weight_decay=5e-4, nesterov=True)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode = 'min', factor=0.1, verbose = True)

train_losses_ = []
train_accuracies_ = []
valid_losses_ = []
valid_accuracies_ = []

epochs = 50
for epoch in range(epochs):
    print(f"\n\tEpoch: {epoch}")
    
    train_loss, train_accuracy, val_loss, val_accuracy = train_val(model, criterion, optimizer, 
                                                                   train_loader, val_loader, device)
    train_losses_.append(train_loss)
    train_accuracies_.append(train_accuracy)
    valid_losses_.append(val_loss)
    valid_accuracies_.append(val_accuracy)
    print(f"\tTraining Loss: {round(train_loss, 4)}; Training Accuracy: {round(train_accuracy*100, 4)}%")
    print(f"\tValidation Loss: {round(val_loss, 4)}; Validation Accuracy: {round(val_accuracy*100, 4)}%")
    
test_loss, test_accuracy = test(model, criterion, test_loader, device)
print(f"\n\tTesting Loss: {round(test_loss, 4)}; Testing Accuracy: {round(test_accuracy*100, 4)}%")

Files already downloaded and verified
Files already downloaded and verified
Num Params: 686026


	Epoch: 0
	Training Loss: 3.3281; Training Accuracy: 38.775%
	Validation Loss: 1.285; Validation Accuracy: 52.05%

	Epoch: 1
	Training Loss: 2.307; Training Accuracy: 59.015%
	Validation Loss: 1.2103; Validation Accuracy: 57.3%

	Epoch: 2
	Training Loss: 1.8299; Training Accuracy: 68.195%
	Validation Loss: 0.8918; Validation Accuracy: 68.68%

	Epoch: 3
	Training Loss: 1.5868; Training Accuracy: 72.385%
	Validation Loss: 1.0391; Validation Accuracy: 66.25%

	Epoch: 4
	Training Loss: 1.4649; Training Accuracy: 74.67%
	Validation Loss: 0.8468; Validation Accuracy: 70.08%

	Epoch: 5
	Training Loss: 1.3877; Training Accuracy: 76.135%
	Validation Loss: 0.8316; Validation Accuracy: 71.57%

	Epoch: 6
	Training Loss: 1.3475; Training Accuracy: 76.855%
	Validation Loss: 0.9354; Validation Accuracy: 68.56%

	Epoch: 7
	Training Loss: 1.318; Training Accuracy: 77.3425%
	Validation Loss: 0.8045; Validati

In [11]:
batch_size = 32

transform = torchvision.transforms.Compose(
    [torchvision.transforms.ToTensor(), torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

trainset = torchvision.datasets.CIFAR10(root = './data', train = True, download = True, transform = transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = batch_size, shuffle = True)

testset = torchvision.datasets.CIFAR10(root = './data', train = False, download = True, transform = transform)
testloader = torch.utils.data.DataLoader(testset, batch_size = batch_size, shuffle = False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

model = ResNet14(num_classes=10)
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay= 0.0001, nesterov = True)


train_losses_ = []
train_accuracies_ = []
test_losses_ = []
test_accuracies_ = []

epochs = 20
for epoch in range(epochs):
    print(f"\n\tEpoch: {epoch}")
    
    train_loss, train_accuracy = train(model, criterion, optimizer, trainloader, device)
    train_losses_.append(train_loss)
    train_accuracies_.append(train_accuracy)
    print(f"\tTraining Loss: {round(train_loss, 4)}; Training Accuracy: {round(train_accuracy*100, 4)}%")
    
    test_loss, test_accuracy = test(model, criterion, testloader, device)
    test_losses_.append(test_loss)
    test_accuracies_.append(test_accuracy)
    print(f"\tTesting Loss: {round(test_loss, 4)}; Testing Accuracy: {round(test_accuracy*100, 4)}%")

Files already downloaded and verified
Files already downloaded and verified


NameError: name 'device' is not defined

EXPERIMENT 2

In [55]:
batch_size = 32

transform = torchvision.transforms.Compose(
    [torchvision.transforms.ToTensor(), torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

trainset = torchvision.datasets.CIFAR10(root = './data', train = True, download = True, transform = transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = batch_size, shuffle = True)

testset = torchvision.datasets.CIFAR10(root = './data', train = False, download = True, transform = transform)
testloader = torch.utils.data.DataLoader(testset, batch_size = batch_size, shuffle = False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

model = ResNet(BasicBlock, [2, 1, 1, 2])
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay= 0.0001, nesterov = True)


train_losses_ = []
train_accuracies_ = []
test_losses_ = []
test_accuracies_ = []

epochs = 15
for epoch in range(epochs):
    print(f"\n\tEpoch: {epoch}")
    
    train_loss, train_accuracy = train(model, criterion, optimizer, trainloader, device)
    train_losses_.append(train_loss)
    train_accuracies_.append(train_accuracy)
    print(f"\tTraining Loss: {round(train_loss, 4)}; Training Accuracy: {round(train_accuracy*100, 4)}%")
    
    test_loss, test_accuracy = test(model, criterion, testloader, device)
    test_losses_.append(test_loss)
    test_accuracies_.append(test_accuracy)
    print(f"\tTesting Loss: {round(test_loss, 4)}; Testing Accuracy: {round(test_accuracy*100, 4)}%")

Files already downloaded and verified
Files already downloaded and verified

	Epoch: 0
	Training Loss: 2.4646; Training Accuracy: 55.228%
	Testing Loss: 0.9236; Testing Accuracy: 67.7%

	Epoch: 1
	Training Loss: 1.5164; Training Accuracy: 73.222%
	Testing Loss: 0.8025; Testing Accuracy: 72.45%

	Epoch: 2
	Training Loss: 1.1271; Training Accuracy: 80.128%
	Testing Loss: 0.6234; Testing Accuracy: 78.53%

	Epoch: 3
	Training Loss: 0.8652; Training Accuracy: 84.866%
	Testing Loss: 0.5852; Testing Accuracy: 79.98%

	Epoch: 4
	Training Loss: 0.6492; Training Accuracy: 88.67%
	Testing Loss: 0.5992; Testing Accuracy: 79.68%

	Epoch: 5
	Training Loss: 0.4686; Training Accuracy: 91.952%
	Testing Loss: 0.6296; Testing Accuracy: 80.52%

	Epoch: 6
	Training Loss: 0.3313; Training Accuracy: 94.288%
	Testing Loss: 0.6194; Testing Accuracy: 81.73%

	Epoch: 7
	Training Loss: 0.2268; Training Accuracy: 96.092%
	Testing Loss: 0.6562; Testing Accuracy: 81.97%

	Epoch: 8
	Training Loss: 0.168; Training Acc

In [58]:
model = ResNet(BasicBlock, [2, 1, 1, 1])
sum(p.numel() for p in model.parameters() if p.requires_grad)

4977226

EXPERIMENT 2

In [60]:
batch_size = 32

transform = torchvision.transforms.Compose(
    [torchvision.transforms.ToTensor(), torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

trainset = torchvision.datasets.CIFAR10(root = './data', train = True, download = True, transform = transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = batch_size, shuffle = True)

testset = torchvision.datasets.CIFAR10(root = './data', train = False, download = True, transform = transform)
testloader = torch.utils.data.DataLoader(testset, batch_size = batch_size, shuffle = False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

model = ResNet(BasicBlock, [2, 1, 1, 1])
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.05, momentum=0.9, weight_decay= 0.0001, nesterov = True)


train_losses_ = []
train_accuracies_ = []
test_losses_ = []
test_accuracies_ = []

epochs = 50
for epoch in range(epochs):
    print(f"\n\tEpoch: {epoch}")
    
    train_loss, train_accuracy = train(model, criterion, optimizer, trainloader, device)
    train_losses_.append(train_loss)
    train_accuracies_.append(train_accuracy)
    print(f"\tTraining Loss: {round(train_loss, 4)}; Training Accuracy: {round(train_accuracy*100, 4)}%")
    
    test_loss, test_accuracy = test(model, criterion, testloader, device)
    test_losses_.append(test_loss)
    test_accuracies_.append(test_accuracy)
    print(f"\tTesting Loss: {round(test_loss, 4)}; Testing Accuracy: {round(test_accuracy*100, 4)}%")

Files already downloaded and verified
Files already downloaded and verified

	Epoch: 0
	Training Loss: 2.7886; Training Accuracy: 49.124%
	Testing Loss: 1.0763; Testing Accuracy: 61.07%

	Epoch: 1
	Training Loss: 1.6935; Training Accuracy: 70.008%
	Testing Loss: 0.7472; Testing Accuracy: 73.9%

	Epoch: 2
	Training Loss: 1.2411; Training Accuracy: 78.266%
	Testing Loss: 0.6213; Testing Accuracy: 78.3%

	Epoch: 3
	Training Loss: 0.9759; Training Accuracy: 82.936%
	Testing Loss: 0.571; Testing Accuracy: 80.48%

	Epoch: 4
	Training Loss: 0.7756; Training Accuracy: 86.46%
	Testing Loss: 0.5698; Testing Accuracy: 81.09%

	Epoch: 5
	Training Loss: 0.5975; Training Accuracy: 89.464%
	Testing Loss: 0.5472; Testing Accuracy: 81.65%

	Epoch: 6
	Training Loss: 0.4597; Training Accuracy: 91.982%
	Testing Loss: 0.6241; Testing Accuracy: 80.55%

	Epoch: 7
	Training Loss: 0.3635; Training Accuracy: 93.704%
	Testing Loss: 0.7156; Testing Accuracy: 79.21%

	Epoch: 8
	Training Loss: 0.3018; Training Accu

KeyboardInterrupt: 